![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20XlnetForSequenceClassification.ipynb)

## Import XLNetForSequenceClassification models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is available only in `Spark NLP 3.4.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import XLNet models trained/fine-tuned for sequence classification via `XLNetForSequenceClassification` or `TFXLNetForSequenceClassification`. These models are usually under `Text Classification` category and have `xlnet` in their labels
- Reference: [TFXLNetForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/xlnet#transformers.TFXLNetForSequenceClassification)
- Some [example models](https://huggingface.co/models?filter=xlnet&pipeline_tag=text-classification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.4.4` version and Transformers on `4.15.0`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- XLNet uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q transformers==4.15.0 tensorflow==2.4.4 sentencepiece

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [mohsenfayyaz/xlnet-base-cased-toxicity](https://huggingface.co/mohsenfayyaz/xlnet-base-cased-toxicity) model from HuggingFace as an example
- In addition to `TFXLNetForSequenceClassification` we also need to save the `XLNetTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import TFXLNetForSequenceClassification, XLNetTokenizer 

MODEL_NAME = 'mohsenfayyaz/xlnet-base-cased-toxicity'

tokenizer = XLNetTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFXLNetForSequenceClassification.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFXLNetForSequenceClassification.from_pretrained(MODEL_NAME, from_pt=True)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

try downloading TF weights


404 Client Error: Not Found for url: https://huggingface.co/mohsenfayyaz/xlnet-base-cased-toxicity/resolve/main/tf_model.h5


try downloading PyTorch weights


All PyTorch model weights were used when initializing TFXLNetForSequenceClassification.

All the weights of TFXLNetForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetForSequenceClassification for predictions without further training.


INFO:tensorflow:Assets written to: ./mohsenfayyaz/xlnet-base-cased-toxicity/saved_model/1/assets


INFO:tensorflow:Assets written to: ./mohsenfayyaz/xlnet-base-cased-toxicity/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 458440
-rw-rw-r-- 1 maziyar maziyar      1004 Dec 28 12:15 config.json
drwxr-xr-x 3 maziyar maziyar      4096 Dec 28 12:15 saved_model
-rw-rw-r-- 1 maziyar maziyar 469430584 Dec 28 12:15 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 6252
drwxr-xr-x 2 maziyar maziyar    4096 Dec 28 12:15 assets
-rw-rw-r-- 1 maziyar maziyar 6392972 Dec 28 12:15 saved_model.pb
drwxr-xr-x 2 maziyar maziyar    4096 Dec 28 12:15 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 788
-rw-rw-r-- 1 maziyar maziyar    291 Dec 28 12:14 special_tokens_map.json
-rw-rw-r-- 1 maziyar maziyar 798011 Dec 28 12:14 spiece.model
-rw-rw-r-- 1 maziyar maziyar    558 Dec 28 12:14 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `spiece.model` file from the tokenizer
- all we need is to copy `spiece.model` file into `saved_model/1/assets` which Spark NLP will look for
- in addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [ ]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's copy spiece.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/spiece.model {asset_path}

In [ ]:
# get label2id dictionary 
labels = model.config.id2label
# sort the dictionary based on the id
labels = [value for key,value in sorted(labels.items(), reverse=False)]

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [ ]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 784
-rw-rw-r-- 1 maziyar maziyar     15 Dec 28 12:15 labels.txt
-rw-rw-r-- 1 maziyar maziyar 798011 Dec 28 12:15 spiece.model


## Import and Save XlnetForSequenceClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `XlnetForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlnetForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [ ]:
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.sql.types import StringType, StructType, StructField

from pyspark.sql import SparkSession

# spark = sparknlp.start(gpu=True)

spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","40g")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars", "./spark-nlp-assembly-3.4.0.jar")\
    .getOrCreate()


In [ ]:
from sparknlp.annotator import *

sequenceClassifier = XlnetForSequenceClassification\
  .loadSavedModel('{}/saved_model/1'.format(MODEL_NAME), spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your XlnetForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 465188
drwxr-xr-x 4 maziyar maziyar      4096 Dec 28 12:15 fields
drwxr-xr-x 2 maziyar maziyar      4096 Dec 28 12:15 metadata
-rw-r--r-- 1 maziyar maziyar 475542482 Dec 28 12:15 xlnet_classification_tensorflow
-rw-r--r-- 1 maziyar maziyar    798011 Dec 28 12:15 xlnet_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XlnetForSequenceClassification model 😊 

In [ ]:
sequenceClassifier_loaded = XlnetForSequenceClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [ ]:
sequenceClassifier_loaded.getClasses()

['Non-Toxic', 'Toxic']

This is how you can use your loaded classifier model in Spark pipeline:

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    sequenceClassifier_loaded    
])

# couple of simple examples
example = spark.createDataFrame([["I love you!"], ['I feel lucky to be here.'], ['I hate her!']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "class.result").show()

+--------------------+-----------+
|                text|     result|
+--------------------+-----------+
|         I love you!|[Non-Toxic]|
|I feel lucky to b...|[Non-Toxic]|
|         I hate her!|    [Toxic]|
+--------------------+-----------+



That's it! You can now go wild and use hundreds of `XlnetForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀 
